In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
cd ..

/Users/stein/repos/privacy-code-tools


In [9]:
import django
django.setup()
from coder.api.models import (
    Assignment,
    AssignmentType,
    Coding,
    CodingInstance,
    Policy,
    PolicyInstance,
    Project,
    RawPolicyInstance,
    TimingSession,
)
from coder.api import document_cleaner

In [28]:

us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}

In [29]:
DOCUMENTS_DIR = "/Users/stein/Dropbox/_Law/Documents_and_Adminitrata/_DATA/scrapers/STATE_DBNR/"

In [30]:
from markdown import markdown

In [31]:
import sys, os

In [33]:
def process_state(fn):
    state_name = fn.split(".")[0].replace("-", " ").title()
    print(state_name)
    with open(f"{DOCUMENTS_DIR}/{fn}") as f:
        raw_md = f.read()
    sections = raw_md.split("\n## ")
    all_content = []
    for idx, section in enumerate(sections):
        citation, *contents = section.split("\n")
        if idx == 0:
            citation = citation.split("## ")[1]
        content = '\n'.join(contents)
        title = [l[:-3] for l in contents if l.endswith("```")]
        title = title[0] if title else citation
        all_content.append({
            "content": document_cleaner.to_coding_doc(markdown(f"`{citation}`\n\n{content}")),
            "ordinal": citation.split("§")[-1].strip().split(" ")[0].strip(),
            "title": title,
        })
    policy = Policy.objects.create(**{
        "project": 4,
        "company_name": state_name,
        "site_name": us_state_to_abbrev.get(state_name, state_name),
    })
    policy.save()
    policy_instance = PolicyInstance.objects.create(**{
        "project": 4,
        "content": all_content,
        "policy_id": policy.id,
    })
    policy_instance.save()

for dirpath, dir_list, filenames in os.walk(DOCUMENTS_DIR):
    for fn in sorted(filenames):
        if fn.endswith("md"):
            process_state(fn)
    break

Alabama
Alaska
Arizona
Arkansas
California
Colorado
Connecticut
Delaware
Florida
Georgia
Hawaii
Idaho
Illinois
Indiana
Iowa
Kansas
Kentucky
Louisiana
Maine
Maryland
Massachusetts
Michigan
Minnesota
Mississippi
Missouri
Montana
Nebraska
Nevada
New Hampshire
New Jersey
New Mexico
New York
North Carolina
North Dakota
Ohio
Oklahoma
Oregon
Pennsylvania
Rhode Island
South Carolina
South Dakota
Tennessee
Texas
Utah
Vermont
Virginia
Washington
West Virginia
Wisconsin
Wyoming
